In [1]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("CUDA Available:", cuda_available)

# If available, display GPU details
if cuda_available:
    print("Number of GPU devices:", torch.cuda.device_count())
    print("Current GPU device index:", torch.cuda.current_device())
    print("Current GPU device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available. PyTorch is running on CPU.")

CUDA Available: True
Number of GPU devices: 1
Current GPU device index: 0
Current GPU device name: NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:

import argparse
import os
import sys
import time
from collections import OrderedDict
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from skimage import io
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from tensorboardX import SummaryWriter
#from dataset import *
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm

import cfg
import function
from conf import settings
#from models.discriminatorlayer import discriminator
from dataset import *
from utils import *

C:\Users\geyan\.conda\envs\monai-dev\Lib\site-packages\ignite\handlers\checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
Importing from timm.models.layers is deprecated, please import via timm.layers
Importing from timm.models.registry is deprecated, please import via timm.models


Unknown arguments: ['-f', 'C:\\Users\\geyan\\AppData\\Roaming\\jupyter\\runtime\\kernel-eea0e018-f342-4b8d-b9e7-a4dcfacc1b55.json']
Unknown arguments: ['-f', 'C:\\Users\\geyan\\AppData\\Roaming\\jupyter\\runtime\\kernel-eea0e018-f342-4b8d-b9e7-a4dcfacc1b55.json']


`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.


In [3]:
# import os
# import SimpleITK as sitk

# # 定义源文件夹和目标文件夹
# source_folder1 = "./data_test/Dataset/images"
# source_folder2 = "./data_test/Dataset/labels"
# target_folder1 = "./data/Dataset/images"
# target_folder2 = "./data/Dataset/labels"

# # 定义处理函数
# def process_files(source_folder, target_folder):
#     if not os.path.exists(target_folder):
#         os.makedirs(target_folder)

#     for file_name in os.listdir(source_folder):
#         if not file_name.endswith(".mha"):  # 忽略非MHA文件
#             continue
        
#         # 获取完整路径
#         input_path = os.path.join(source_folder, file_name)
#         image = sitk.ReadImage(input_path)
#         array = sitk.GetArrayFromImage(image)

#         # 打印原图尺寸
#         print(f"Processing file: {file_name}")
#         print(f"Original image dimensions (D, H, W): {array.shape}")
        
#         # 检查文件名模式并生成新文件名
#         file_base, file_ext = os.path.splitext(file_name)
#         if file_base.endswith("_0000"):
#             name_base = file_base[:-5]  # 去掉 "_0000"
#             template = f"{name_base}_x_0000{file_ext}"
#         else:
#             name_base = file_base
#             template = f"{name_base}_x{file_ext}"
        
#         # 获取深度并切分
#         depth, height, width = array.shape
#         slice_size = depth // 8
#         for idx in range(8):
#             start_idx = idx * slice_size
#             end_idx = (idx + 1) * slice_size if idx < 7 else depth
#             slice_array = array[start_idx:end_idx]

#             # 转换为SimpleITK图像
#             slice_image = sitk.GetImageFromArray(slice_array)

#             # 复制元数据（方向、原点、间距）
#             slice_image.SetDirection(image.GetDirection())
#             slice_image.SetOrigin(image.GetOrigin())
#             spacing = list(image.GetSpacing())
#             spacing[2] = spacing[2] * (slice_array.shape[0] / depth)  # 更新Z方向间距
#             slice_image.SetSpacing(spacing)

#             # 打印切片尺寸
#             print(f"Slice {idx + 1} dimensions (D, H, W): {slice_array.shape}")

#             # 动态生成目标文件名
#             output_name = template.replace("_x", f"_x{idx + 1}")
#             output_path = os.path.join(target_folder, output_name)

#             # 保存到目标文件夹
#             sitk.WriteImage(slice_image, output_path)
#             print(f"Saved slice {idx + 1} to {output_path}")

# # 分别处理两个文件夹
# process_files(source_folder1, target_folder1)
# process_files(source_folder2, target_folder2)


In [4]:
args = cfg.parse_args()

GPUdevice = torch.device('cuda', args.gpu_device)

net = get_network(args, args.net, use_gpu=args.gpu, gpu_device=GPUdevice, distribution = args.distributed)
if args.pretrain:
    weights = torch.load(args.pretrain)
    net.load_state_dict(weights,strict=False)

optimizer = optim.Adam(net.parameters(), lr=args.lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5) #learning rate decay

'''load pretrained model'''
if args.weights != 0:
    print(f'=> resuming from {args.weights}')
    assert os.path.exists(args.weights)
    checkpoint_file = os.path.join(args.weights)
    assert os.path.exists(checkpoint_file)
    loc = 'cuda:{}'.format(args.gpu_device)
    checkpoint = torch.load(checkpoint_file, map_location=loc)
    start_epoch = checkpoint['epoch']
    best_tol = checkpoint['best_tol']

    net.load_state_dict(checkpoint['state_dict'],strict=False)
    # optimizer.load_state_dict(checkpoint['optimizer'], strict=False)

    args.path_helper = checkpoint['path_helper']
    logger = create_logger(args.path_helper['log_path'])
    print(f'=> loaded checkpoint {checkpoint_file} (epoch {start_epoch})')

args.path_helper = set_log_dir('logs', args.exp_name)
logger = create_logger(args.path_helper['log_path'])
logger.info(args)

nice_train_loader, nice_test_loader = get_dataloader(args)
# 打印加载后的数据张量大小
#for batch_idx, data in enumerate(nice_train_loader):
#    imgs, masks = data['image'], data['label']
#    print(f"批次 {batch_idx} 的图像张量大小: {imgs.shape}")
#    print(f"批次 {batch_idx} 的标签张量大小: {masks.shape}")
#    break  # 仅打印第一个批次


Unknown arguments: ['-f', 'C:\\Users\\geyan\\AppData\\Roaming\\jupyter\\runtime\\kernel-eea0e018-f342-4b8d-b9e7-a4dcfacc1b55.json']


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
Namespace(net='sam', baseline='unet', encoder='default', seg_net='transunet', mod='sam_adpt', exp_name='msa_test_isic', type='map', vis=10, reve

In [5]:
'''checkpoint path and tensorboard'''
# iter_per_epoch = len(Glaucoma_training_loader)
checkpoint_path = os.path.join(settings.CHECKPOINT_PATH, args.net, settings.TIME_NOW)
#use tensorboard
if not os.path.exists(settings.LOG_DIR):
    os.mkdir(settings.LOG_DIR)
writer = SummaryWriter(log_dir=os.path.join(
        settings.LOG_DIR, args.net, settings.TIME_NOW))
# input_tensor = torch.Tensor(args.b, 3, 256, 256).cuda(device = GPUdevice)
# writer.add_graph(net, Variable(input_tensor, requires_grad=True))

#create checkpoint folder to save model
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, '{net}-{epoch}-{type}.pth')

In [6]:
'''begain training'''
best_acc = 0.0
best_tol = 1e4
best_dice = 0.0

# Print the total number of parameters
num_params = sum(p.numel() for p in net.parameters())
logger.info(f"Total number of parameters: {num_params}")
print(f"Total number of parameters: {num_params}")

def print_model_details(model):
    print("Model Structure:")
    print(model)
    print("\nDetailed Layer-wise Parameters:")
    total_params = 0
    for name, param in model.named_parameters():
        layer_params = param.numel()
        total_params += layer_params
        print(f"Layer: {name} | Parameters: {layer_params}")
    print(f"\nTotal Parameters: {total_params}")

# print_model_details(net)


for epoch in range(settings.EPOCH):
    # if epoch and epoch < 5:
    #     if args.dataset != 'REFUGE':
    #         tol, (eiou, edice) = function.validation_sam(args, nice_test_loader, epoch, net, writer)
    #         logger.info(f'Total score: {tol}, IOU: {eiou}, DICE: {edice} || @ epoch {epoch}.')
    #     else:
    #         tol, (eiou_cup, eiou_disc, edice_cup, edice_disc) = function.validation_sam(args, nice_test_loader, epoch, net, writer)
    #         logger.info(f'Total score: {tol}, IOU_CUP: {eiou_cup}, IOU_DISC: {eiou_disc}, DICE_CUP: {edice_cup}, DICE_DISC: {edice_disc} || @ epoch {epoch}.')
    if epoch and epoch < 30:
        if args.dataset != 'REFUGE':
            # 接收返回的五个值
            tol, avg_iou, avg_dice, iou_per_class, dice_per_class = function.validation_sam(args, nice_test_loader, epoch, net, writer)
            logger.info(f'Total loss: {tol}, Average IOU: {avg_iou}, Average Dice: {avg_dice}')
            # 如果需要展示每个类别的指标
            for i, (iou, dice) in enumerate(zip(iou_per_class, dice_per_class)):
                logger.info(f'Class {i}: IOU={iou:.4f}, Dice={dice:.4f}')
    net.train()
    time_start = time.time()
    
    loss = function.train_sam(args, net, optimizer, nice_train_loader, epoch, writer, vis = args.vis)
    logger.info(f'Train loss: {loss} || @ epoch {epoch}.')
    time_end = time.time()
    print('time_for_training ', time_end - time_start)

    net.eval()
    if epoch and epoch % args.val_freq == 0 or epoch == settings.EPOCH-1:
        # if args.dataset != 'REFUGE':
        #     tol, (eiou, edice) = function.validation_sam(args, nice_test_loader, epoch, net, writer)
        #     logger.info(f'Total score: {tol}, IOU: {eiou}, DICE: {edice} || @ epoch {epoch}.')
        # else:
        #     tol, (eiou_cup, eiou_disc, edice_cup, edice_disc) = function.validation_sam(args, nice_test_loader, epoch, net, writer)
        #     logger.info(f'Total score: {tol}, IOU_CUP: {eiou_cup}, IOU_DISC: {eiou_disc}, DICE_CUP: {edice_cup}, DICE_DISC: {edice_disc} || @ epoch {epoch}.')
        if args.dataset != 'REFUGE':
            # 接收返回的五个值
            tol, avg_iou, avg_dice, iou_per_class, dice_per_class = function.validation_sam(args, nice_test_loader, epoch, net, writer)
            logger.info(f'Total loss: {tol}, Average IOU: {avg_iou}, Average Dice: {avg_dice}')
            # 如果需要展示每个类别的指标
            for i, (iou, dice) in enumerate(zip(iou_per_class, dice_per_class)):
                logger.info(f'Class {i}: IOU={iou:.4f}, Dice={dice:.4f}')
        if args.distributed != 'none':
            sd = net.module.state_dict()
        else:
            sd = net.state_dict()

        if avg_dice > best_dice:
            best_tol = tol
            is_best = True

            save_checkpoint({
            'epoch': epoch + 1,
            'model': args.net,
            'state_dict': sd,
            'best_tol': best_dice,
            'path_helper': args.path_helper,
        }, is_best, args.path_helper['ckpt_path'], filename="best_dice_checkpoint.pth")
        else:
            is_best = False

writer.close()

Total number of parameters: 110652157


Total number of parameters: 110652157


Epoch 0: 100%|███████████████████████████████████████████████████| 353/353 [12:49<00:00,  2.18s/img, loss (batch)=1.09]
Train loss: 1.087186336517334 || @ epoch 0.


time_for_training  769.669754743576


Total loss: 0.0, Average IOU: 0.7115784967888374, Average Dice: 0.5946507542109003                                     
Class 0: IOU=0.9242, Dice=0.9602
Class 1: IOU=0.1745, Dice=0.2750
Class 2: IOU=0.0339, Dice=0.0558
Class 3: IOU=0.4503, Dice=0.4000
Class 4: IOU=0.4570, Dice=0.4040
Class 5: IOU=0.4528, Dice=0.3166
Class 6: IOU=0.9073, Dice=0.9007
Class 7: IOU=0.1200, Dice=0.2067
Class 8: IOU=0.9710, Dice=0.9536
Class 9: IOU=0.8534, Dice=0.6526
Class 10: IOU=0.9706, Dice=0.9669
Class 11: IOU=0.3889, Dice=0.1201
Class 12: IOU=0.5378, Dice=0.2148
Class 13: IOU=0.3512, Dice=0.1222
Class 14: IOU=0.7542, Dice=0.6002
Class 15: IOU=0.6949, Dice=0.4087
Class 16: IOU=0.5807, Dice=0.2847
Class 17: IOU=0.5097, Dice=0.2347
Class 18: IOU=0.9123, Dice=0.9007
Class 19: IOU=1.0000, Dice=1.0000
Class 20: IOU=1.0000, Dice=1.0000
Class 21: IOU=0.6721, Dice=0.4008
Class 22: IOU=0.7363, Dice=0.5404
Class 23: IOU=0.4205, Dice=0.0988
Class 24: IOU=0.7564, Dice=0.5073
Class 25: IOU=0.8183, Dice=0.6552
Class 

...

Total loss: 0.0, Average IOU: 0.8099090610535777, Average Dice: 0.7732047724480532                                     
Class 0: IOU=0.7176, Dice=0.8308
Class 1: IOU=0.0931, Dice=0.0596
Class 2: IOU=0.8026, Dice=0.7881
Class 3: IOU=0.4507, Dice=0.4040
Class 4: IOU=0.4570, Dice=0.4040
Class 5: IOU=0.9065, Dice=0.8940
Class 6: IOU=0.9073, Dice=0.9007
Class 7: IOU=0.6935, Dice=0.8045
Class 8: IOU=0.9710, Dice=0.9536
Class 9: IOU=0.9143, Dice=0.8808
Class 10: IOU=0.9706, Dice=0.9669
Class 11: IOU=0.7446, Dice=0.6887
Class 12: IOU=0.7690, Dice=0.7020
Class 13: IOU=0.7405, Dice=0.6887
Class 14: IOU=0.7781, Dice=0.7020
Class 15: IOU=0.8220, Dice=0.7550
Class 16: IOU=0.8146, Dice=0.7550
Class 17: IOU=0.8257, Dice=0.7815
Class 18: IOU=0.9123, Dice=0.9007
Class 19: IOU=1.0000, Dice=1.0000
Class 20: IOU=1.0000, Dice=1.0000
Class 21: IOU=0.7537, Dice=0.6954
Class 22: IOU=0.7637, Dice=0.6887
Class 23: IOU=0.7471, Dice=0.7020
Class 24: IOU=0.8096, Dice=0.7351
Class 25: IOU=0.8394, Dice=0.7815
Class 

time_for_training  748.5514724254608


Total loss: 0.0, Average IOU: 0.8112418878139281, Average Dice: 0.7742037110182703                                     
Class 0: IOU=0.7136, Dice=0.8273
Class 1: IOU=0.0931, Dice=0.0596
Class 2: IOU=0.8026, Dice=0.7881
Class 3: IOU=0.4507, Dice=0.4040
Class 4: IOU=0.4570, Dice=0.4040
Class 5: IOU=0.9065, Dice=0.8940
Class 6: IOU=0.9073, Dice=0.9007
Class 7: IOU=0.7628, Dice=0.8570
Class 8: IOU=0.9710, Dice=0.9536
Class 9: IOU=0.9143, Dice=0.8808
Class 10: IOU=0.9706, Dice=0.9669
Class 11: IOU=0.7446, Dice=0.6887
Class 12: IOU=0.7690, Dice=0.7020
Class 13: IOU=0.7405, Dice=0.6887
Class 14: IOU=0.7781, Dice=0.7020
Class 15: IOU=0.8220, Dice=0.7550
Class 16: IOU=0.8146, Dice=0.7550
Class 17: IOU=0.8257, Dice=0.7815
Class 18: IOU=0.9123, Dice=0.9007
Class 19: IOU=1.0000, Dice=1.0000
Class 20: IOU=1.0000, Dice=1.0000
Class 21: IOU=0.7537, Dice=0.6954
Class 22: IOU=0.7637, Dice=0.6887
Class 23: IOU=0.7471, Dice=0.7020
Class 24: IOU=0.8096, Dice=0.7351
Class 25: IOU=0.8394, Dice=0.7815
Class 

time_for_training  761.2223041057587


MemoryError: Unable to allocate 196. MiB for an array with shape (16, 49, 256, 256) and data type float32